# Recurrent DQN

In this notebook, we'll build a neural network that can learn to play games through reinforcement learning. More specifically, we'll use Q-learning to train an agent to play a game called [Cart-Pole](https://gym.openai.com/envs/CartPole-v0). In this game, a freely swinging pole is attached to a cart. The cart can move to the left and right, and the goal is to keep the pole upright as long as possible.

![Cart-Pole](assets/cart-pole.jpg)

We can simulate this game using [OpenAI Gym](https://gym.openai.com/). First, let's check out how OpenAI Gym works. Then, we'll get into training an agent to play the Cart-Pole game.

In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


>**Note:** Make sure you have OpenAI Gym cloned into the same directory with this notebook. I've included `gym` as a submodule, so you can run `git submodule --init --recursive` to pull the contents into the `gym` repo.

>**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import gym

# Create the Cart-Pole game environment
# env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

/home/arasdar/anaconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


We interact with the simulation through `env`. To show the simulation running, you can use `env.render()` to render one frame. Passing in an action as an integer to `env.step` will generate the next step in the simulation.  You can see how many actions are possible from `env.action_space` and to get a random action you can use `env.action_space.sample()`. This is general to all Gym games. In the Cart-Pole game, there are two possible actions, moving the cart left or right. So there are two actions we can take, encoded as 0 and 1.

Run the code below to watch the simulation run.

In [3]:
# import numpy as np
# state = env.reset()
# for _ in range(10):
#     # env.render()
#     action = env.action_space.sample()
#     next_state, reward, done, info = env.step(action) # take a random action
#     #print('state, action, next_state, reward, done, info:', state, action, next_state, reward, done, info)
#     state = next_state
#     if done:
#         state = env.reset()

To shut the window showing the simulation, use `env.close()`.

If you ran the simulation above, we can look at the rewards:

In [4]:
# print(rewards[-20:])
# print(np.array(rewards).shape, np.array(states).shape, np.array(actions).shape, np.array(dones).shape)
# print(np.array(rewards).dtype, np.array(states).dtype, np.array(actions).dtype, np.array(dones).dtype)
# print(np.max(np.array(actions)), np.min(np.array(actions)))
# print((np.max(np.array(actions)) - np.min(np.array(actions)))+1)
# print(np.max(np.array(rewards)), np.min(np.array(rewards)))
# print(np.max(np.array(states)), np.min(np.array(states)))

The game resets after the pole has fallen past a certain angle. For each frame while the simulation is running, it returns a reward of 1.0. The longer the game runs, the more reward we get. Then, our network's goal is to maximize the reward by keeping the pole vertical. It will do this by moving the cart to the left and the right.

## Q-Network

We train our Q-learning agent using the Bellman Equation:

$$
Q(s, a) = r + \gamma \max{Q(s', a')}
$$

where $s$ is a state, $a$ is an action, and $s'$ is the next state from state $s$ and action $a$.

Before we used this equation to learn values for a Q-_table_. However, for this game there are a huge number of states available. The state has four values: the position and velocity of the cart, and the position and velocity of the pole. These are all real-valued numbers, so ignoring floating point precisions, you practically have infinite states. Instead of using a table then, we'll replace it with a neural network that will approximate the Q-table lookup function.

<img src="assets/deep-q-learning.png" width=450px>

Now, our Q value, $Q(s, a)$ is calculated by passing in a state to the network. The output will be Q-values for each available action, with fully connected hidden layers.

<img src="assets/q-network.png" width=550px>


As I showed before, we can define our targets for training as $\hat{Q}(s,a) = r + \gamma \max{Q(s', a')}$. Then we update the weights by minimizing $(\hat{Q}(s,a) - Q(s,a))^2$. 

For this Cart-Pole game, we have four inputs, one for each value in the state, and two outputs, one for each action. To get $\hat{Q}$, we'll first choose an action, then simulate the game using that action. This will get us the next state, $s'$, and the reward. With that, we can calculate $\hat{Q}$ then pass it back into the $Q$ network to run the optimizer and update the weights.

Below is my implementation of the Q-network. I used two fully connected layers with ReLU activations. Two seems to be good enough, three might be better. Feel free to try it out.

In [6]:
def model_input(state_size, hidden_size, batch_size=1):
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    targetQs = tf.placeholder(tf.float32, [None], name='targetQs')
    # RNN
    cell = tf.nn.rnn_cell.GRUCell(hidden_size)
    #cell = tf.nn.rnn_cell.LSTMCell(hidden_size)
    cells = tf.nn.rnn_cell.MultiRNNCell([cell], state_is_tuple=True)
    initial_state = cells.zero_state(batch_size, tf.float32)
    is_training = tf.placeholder(dtype=tf.bool, shape=[], name='is_training')
    return states, actions, targetQs, cells, initial_state, is_training

In [7]:
# RNN generator or sequence generator
def generator(states, action_size, initial_state, cells, hidden_size, reuse=False, is_training=True): 
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        #inputs = tf.layers.dense(inputs=states, units=hidden_size)
        inputs = tf.contrib.layers.fully_connected(inputs=states, num_outputs=hidden_size, activation_fn=None)
        #inputs = tf.layers.batch_normalization(inputs=inputs, training=is_training)
        #inputs = tf.maximum(0.1*inputs, inputs)
        print(states.shape, inputs.shape)
        
        # with tf.variable_scope('dynamic_rnn_', reuse=tf.AUTO_REUSE):
        # dynamic means adapt to the batch_size and
        # static means can NOT adapt to the batch_size
        inputs_rnn = tf.reshape(inputs, [1, -1, hidden_size]) # NxH -> 1xNxH
        print(inputs_rnn.shape, initial_state)
        outputs_rnn, final_state = tf.nn.dynamic_rnn(cell=cells, inputs=inputs_rnn, initial_state=initial_state)
        #outputs_rnn = tf.layers.batch_normalization(inputs=outputs_rnn, training=is_training)
        #final_state = tf.layers.batch_normalization(inputs=final_state, training=is_training)
        print(outputs_rnn.shape, final_state)
        outputs = tf.reshape(outputs_rnn, [-1, hidden_size]) # 1xNxH -> NxH
        print(outputs.shape)

        # Last fully connected layer
        #logits = tf.layers.dense(inputs=outputs, units=action_size)
        logits = tf.contrib.layers.fully_connected(inputs=outputs, num_outputs=action_size, activation_fn=None)
        print(logits.shape)
        #predictions = tf.nn.softmax(logits)
        
        # logits are the action logits
        return logits, final_state

In [8]:
def model_loss(action_size, hidden_size, states, cells, initial_state, actions, targetQs, is_training):
    actions_logits, final_state = generator(states=states, cells=cells, initial_state=initial_state, 
                                            hidden_size=hidden_size, action_size=action_size, 
                                            is_training=is_training)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    Qs = tf.reduce_max(actions_logits*actions_labels, axis=1)
    loss = tf.reduce_mean(tf.square(Qs - targetQs))
    return actions_logits, final_state, loss

In [9]:
def model_opt(loss, learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    g_vars = [var for var in t_vars if var.name.startswith('generator')]

    # # Optimize MLP/CNN/RNN without clipping grads
    #with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
    opt = tf.train.AdamOptimizer(learning_rate).minimize(loss, var_list=g_vars)

    # # Optimize RNN
    #grads, _ = tf.clip_by_global_norm(t_list=tf.gradients(loss, g_vars), clip_norm=5) # usually around 1-5
    #grads = tf.gradients(loss, g_vars)
    #opt = tf.train.AdamOptimizer(learning_rate).apply_gradients(grads_and_vars=zip(grads, g_vars))
    return opt

In [10]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, learning_rate):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.targetQs, cells, self.initial_state, self.is_training = model_input(
                state_size=state_size, hidden_size=hidden_size)
        
        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.final_state, self.loss = model_loss(
            action_size=action_size, hidden_size=hidden_size, 
            states=self.states, actions=self.actions, 
            targetQs=self.targetQs, cells=cells, initial_state=self.initial_state, is_training=self.is_training)

        # Update the model: backward pass and backprop
        self.opt = model_opt(loss=self.loss, learning_rate=learning_rate)

## Experience replay

Reinforcement learning algorithms can have stability issues due to correlations between states. To reduce correlations when training, we can store the agent's experiences and later draw a random mini-batch of those experiences to train on. 

Here, we'll create a `Memory` object that will store our experiences, our transitions $<s, a, r, s'>$. This memory will have a maxmium capacity, so we can keep newer experiences in memory while getting rid of older experiences. Then, we'll sample a random mini-batch of transitions $<s, a, r, s'>$ and train on those.

Below, I've implemented a `Memory` object. If you're unfamiliar with `deque`, this is a double-ended queue. You can think of it like a tube open on both sides. You can put objects in either side of the tube. But if it's full, adding anything more will push an object out the other side. This is a great data structure to use for the memory buffer.

In [11]:
from collections import deque

class Memory():    
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size)
        self.states = deque(maxlen=max_size)

## Exploration - Exploitation

To learn about the environment and rules of the game, the agent needs to explore by taking random actions. We'll do this by choosing a random action with some probability $\epsilon$ (epsilon).  That is, with some probability $\epsilon$ the agent will make a random action and with probability $1 - \epsilon$, the agent will choose an action from $Q(s,a)$. This is called an **$\epsilon$-greedy policy**.


At first, the agent needs to do a lot of exploring. Later when it has learned more, the agent can favor choosing actions based on what it has learned. This is called _exploitation_. We'll set it up so the agent is more likely to explore early in training, then more likely to exploit later in training.

## Q-Learning training algorithm

Putting all this together, we can list out the algorithm we'll use to train the network. We'll train the network in _episodes_. One *episode* is one simulation of the game. For this game, the goal is to keep the pole upright for 195 frames. So we can start a new episode once meeting that goal. The game ends if the pole tilts over too far, or if the cart moves too far the left or right. When a game ends, we'll start a new episode. Now, to train the agent:

* Initialize the memory $D$
* Initialize the action-value network $Q$ with random weights
* **For** episode = 1, $M$ **do**
  * **For** $t$, $T$ **do**
     * With probability $\epsilon$ select a random action $a_t$, otherwise select $a_t = \mathrm{argmax}_a Q(s,a)$
     * Execute action $a_t$ in simulator and observe reward $r_{t+1}$ and new state $s_{t+1}$
     * Store transition $<s_t, a_t, r_{t+1}, s_{t+1}>$ in memory $D$
     * Sample random mini-batch from $D$: $<s_j, a_j, r_j, s'_j>$
     * Set $\hat{Q}_j = r_j$ if the episode ends at $j+1$, otherwise set $\hat{Q}_j = r_j + \gamma \max_{a'}{Q(s'_j, a')}$
     * Make a gradient descent step with loss $(\hat{Q}_j - Q(s_j, a_j))^2$
  * **endfor**
* **endfor**

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [12]:
# Network parameters
action_size = 2
state_size = 4
hidden_size = 64               # number of units in each Q-network hidden layer
learning_rate = 0.0001         # Q-network learning rate

# Memory parameters
memory_size = 128            # memory capacity - 1000 DQN
batch_size = 128             # experience mini-batch size - 20 DQN
gamma = 0.99                 # future reward discount

In [13]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, hidden_size=hidden_size, state_size=state_size, learning_rate=learning_rate)

# Init the memory
memory = Memory(max_size=batch_size)

(?, 4) (?, 64)
(1, ?, 64) (<tf.Tensor 'MultiRNNCellZeroState/GRUCellZeroState/zeros:0' shape=(1, 64) dtype=float32>,)
(1, ?, 64) (<tf.Tensor 'generator/rnn/while/Exit_3:0' shape=(1, 64) dtype=float32>,)
(?, 64)
(?, 2)


In [14]:
model.initial_state[0]

<tf.Tensor 'MultiRNNCellZeroState/GRUCellZeroState/zeros:0' shape=(1, 64) dtype=float32>

## Populate the memory (exprience memory)

Here I'm re-initializing the simulation and pre-populating the memory. The agent is taking random actions and storing the transitions in memory. This will help the agent with exploring the game.

In [15]:
import numpy as np
state = env.reset()
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    memory.buffer.append([state, action, next_state, reward, float(done)])
    memory.states.append(np.zeros([1, hidden_size])) # gru
    #memory.states.append([np.zeros([1, hidden_size]), np.zeros([1, hidden_size])]) # lstm
    state = next_state
    if done is True:
        # Reseting the env/first state
        state = env.reset()

In [16]:
# # Training
# batch = memory.buffer
# states = np.array([each[0] for each in batch])
# actions = np.array([each[1] for each in batch])
# next_states = np.array([each[2] for each in batch])
# rewards = np.array([each[3] for each in batch])
# dones = np.array([each[4] for each in batch])

In [17]:
memory.states[0].shape, model.initial_state[0].shape # gru
# memory.states[0][1].shape, model.initial_state[0][1].shape #lstm

((1, 64), TensorShape([Dimension(1), Dimension(64)]))

In [18]:
# memory.states[0][0].shape, model.initial_state[0][0].shape

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [19]:
# initial_states = np.array(memory.states)
# initial_states.shape

In [ ]:
saver = tf.train.Saver()
episode_rewards_list, rewards_list, loss_list = [], [], []

# TF session for training
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window
    
    # Training episodes/epochs
    for ep in range(11111):
        total_reward = 0
        loss_batch = []
        state = env.reset()
        initial_state = sess.run(model.initial_state)

        # Training steps/batches
        while True:
            action_logits, final_state = sess.run([model.actions_logits, model.final_state],
                                                  feed_dict = {model.states: state.reshape([1, -1]), 
                                                               model.is_training: False,
                                                               model.initial_state: initial_state})
            action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done)])
            memory.states.append(initial_state)
            total_reward += reward
            state = next_state
            initial_state = final_state

            # Training
            batch = memory.buffer
            states = np.array([each[0] for each in batch])
            actions = np.array([each[1] for each in batch])
            next_states = np.array([each[2] for each in batch])
            rewards = np.array([each[3] for each in batch])
            dones = np.array([each[4] for each in batch])
            initial_states = memory.states
            next_actions_logits = sess.run(model.actions_logits, 
                                           feed_dict = {model.states: next_states,
                                                        model.is_training: False,
                                                        model.initial_state: initial_states[1]})
            nextQs = np.max(next_actions_logits, axis=1) * (1-dones)
            targetQs = rewards + (gamma * nextQs)
            loss, _ = sess.run([model.loss, model.opt], feed_dict = {model.states: states, 
                                                                     model.actions: actions,
                                                                     model.targetQs: targetQs,
                                                                     model.is_training: True,
                                                                     model.initial_state: initial_states[0]})
            # End of training
            loss_batch.append(loss)
            if done is True:
                break
                
        # Outputing: priting out/Potting
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'loss:{:.4f}'.format(np.mean(loss_batch)))
        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        loss_list.append([ep, np.mean(loss_batch)])
        # Break episode/epoch loop
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:9.0000 R:9.0000 loss:1.0467
Episode:1 meanR:9.0000 R:9.0000 loss:0.9682
Episode:2 meanR:9.0000 R:9.0000 loss:0.9560
Episode:3 meanR:9.0000 R:9.0000 loss:0.9286
Episode:4 meanR:8.8000 R:8.0000 loss:0.9060
Episode:5 meanR:8.8333 R:9.0000 loss:0.9004
Episode:6 meanR:8.7143 R:8.0000 loss:0.8640
Episode:7 meanR:8.7500 R:9.0000 loss:0.8014
Episode:8 meanR:8.7778 R:9.0000 loss:0.8661
Episode:9 meanR:8.8000 R:9.0000 loss:0.9917
Episode:10 meanR:8.8182 R:9.0000 loss:0.9834
Episode:11 meanR:8.8333 R:9.0000 loss:1.1289
Episode:12 meanR:8.8462 R:9.0000 loss:1.4240
Episode:13 meanR:8.9286 R:10.0000 loss:1.7760
Episode:14 meanR:8.8667 R:8.0000 loss:2.0664
Episode:15 meanR:8.9375 R:10.0000 loss:2.4236
Episode:16 meanR:8.9412 R:9.0000 loss:2.6084
Episode:17 meanR:9.0000 R:10.0000 loss:3.0851
Episode:18 meanR:9.0000 R:9.0000 loss:3.2646
Episode:19 meanR:8.9500 R:8.0000 loss:3.7522
Episode:20 meanR:8.9048 R:8.0000 loss:3.8816
Episode:21 meanR:8.9091 R:9.0000 loss:4.2070
Episode:22 meanR:

Episode:175 meanR:46.4500 R:32.0000 loss:7.0689
Episode:176 meanR:46.1300 R:28.0000 loss:13.3467
Episode:177 meanR:45.7700 R:41.0000 loss:19.7764
Episode:178 meanR:46.5700 R:119.0000 loss:11.6305
Episode:179 meanR:47.2000 R:106.0000 loss:1.1850
Episode:180 meanR:48.0500 R:122.0000 loss:0.6443
Episode:181 meanR:48.0300 R:33.0000 loss:1.6893
Episode:182 meanR:48.9100 R:134.0000 loss:5.8248
Episode:183 meanR:48.5900 R:18.0000 loss:3.7875
Episode:184 meanR:48.2600 R:18.0000 loss:8.9402
Episode:185 meanR:48.2700 R:17.0000 loss:40.4200
Episode:186 meanR:48.3100 R:18.0000 loss:79.4423
Episode:187 meanR:48.4100 R:24.0000 loss:46.5917
Episode:188 meanR:48.4000 R:16.0000 loss:65.1895
Episode:189 meanR:48.4400 R:18.0000 loss:177.5946
Episode:190 meanR:48.4900 R:18.0000 loss:139.4620
Episode:191 meanR:48.5200 R:20.0000 loss:126.9098
Episode:192 meanR:48.5700 R:20.0000 loss:118.0915
Episode:193 meanR:48.5700 R:16.0000 loss:134.6103
Episode:194 meanR:48.6800 R:21.0000 loss:111.0099
Episode:195 meanR

Episode:340 meanR:285.9300 R:500.0000 loss:2.2083
Episode:341 meanR:284.4600 R:145.0000 loss:55.0653
Episode:342 meanR:285.9800 R:500.0000 loss:2.3889
Episode:343 meanR:288.1400 R:500.0000 loss:15.7232
Episode:344 meanR:288.6700 R:202.0000 loss:38.9608
Episode:345 meanR:293.3700 R:500.0000 loss:4.0650
Episode:346 meanR:293.4800 R:116.0000 loss:61.1797
Episode:347 meanR:296.1900 R:292.0000 loss:4.4649
Episode:348 meanR:301.0000 R:500.0000 loss:5.0194
Episode:349 meanR:305.0900 R:500.0000 loss:15.9677
Episode:350 meanR:308.8000 R:500.0000 loss:15.9677
Episode:351 meanR:312.6800 R:500.0000 loss:16.0498
Episode:352 meanR:313.8600 R:273.0000 loss:29.6934
Episode:353 meanR:317.7300 R:500.0000 loss:7.7973
Episode:354 meanR:317.7300 R:500.0000 loss:5.6743
Episode:355 meanR:320.4700 R:500.0000 loss:15.7821
Episode:356 meanR:324.5100 R:500.0000 loss:15.4527
Episode:357 meanR:327.0500 R:500.0000 loss:15.1766
Episode:358 meanR:329.9100 R:311.0000 loss:15.1078
Episode:359 meanR:329.9600 R:256.0000 

Episode:503 meanR:370.6000 R:293.0000 loss:3.5270
Episode:504 meanR:369.7800 R:235.0000 loss:6.1629
Episode:505 meanR:369.5600 R:201.0000 loss:5.4633
Episode:506 meanR:369.4000 R:291.0000 loss:3.3982
Episode:507 meanR:368.6900 R:366.0000 loss:3.4319
Episode:508 meanR:369.7200 R:500.0000 loss:1.8378
Episode:509 meanR:371.6400 R:500.0000 loss:7.6006
Episode:510 meanR:373.6300 R:500.0000 loss:11.4806
Episode:511 meanR:375.7700 R:500.0000 loss:8.3869
Episode:512 meanR:377.9100 R:500.0000 loss:17.0206
Episode:513 meanR:380.1700 R:500.0000 loss:10.4035
Episode:514 meanR:382.4300 R:500.0000 loss:9.9490
Episode:515 meanR:384.9000 R:500.0000 loss:16.6122
Episode:516 meanR:387.4600 R:500.0000 loss:14.0733
Episode:517 meanR:389.9300 R:500.0000 loss:16.3693
Episode:518 meanR:392.5100 R:500.0000 loss:8.9130
Episode:519 meanR:395.2800 R:500.0000 loss:2.3208
Episode:520 meanR:398.3300 R:500.0000 loss:13.8630
Episode:521 meanR:401.0800 R:500.0000 loss:10.1885
Episode:522 meanR:403.2000 R:500.0000 loss

Episode:665 meanR:464.0200 R:500.0000 loss:13.8148
Episode:666 meanR:464.0200 R:500.0000 loss:16.8639
Episode:667 meanR:464.0200 R:500.0000 loss:15.3410
Episode:668 meanR:464.0200 R:500.0000 loss:18.5413
Episode:669 meanR:459.1100 R:9.0000 loss:74.6169
Episode:670 meanR:454.1900 R:8.0000 loss:135.0093
Episode:671 meanR:449.2800 R:9.0000 loss:180.3061
Episode:672 meanR:444.3700 R:9.0000 loss:213.6908
Episode:673 meanR:439.9900 R:62.0000 loss:174.4552
Episode:674 meanR:439.9900 R:500.0000 loss:16.4388
Episode:675 meanR:439.9900 R:500.0000 loss:20.9716
Episode:676 meanR:439.9900 R:500.0000 loss:19.2024
Episode:677 meanR:439.9900 R:500.0000 loss:17.0914
Episode:678 meanR:439.9900 R:500.0000 loss:16.9162
Episode:679 meanR:439.9900 R:500.0000 loss:15.6760
Episode:680 meanR:439.9900 R:500.0000 loss:16.2475
Episode:681 meanR:439.9900 R:500.0000 loss:16.7801
Episode:682 meanR:439.9900 R:500.0000 loss:16.1077
Episode:683 meanR:439.9900 R:500.0000 loss:16.0893
Episode:684 meanR:439.9900 R:500.000

...

Episode:4381 meanR:403.2000 R:318.0000 loss:0.6368
Episode:4382 meanR:402.3500 R:415.0000 loss:0.8388
Episode:4383 meanR:402.3500 R:500.0000 loss:7.4615
Episode:4384 meanR:401.3300 R:398.0000 loss:35.9788
Episode:4385 meanR:396.9900 R:66.0000 loss:6.1901
Episode:4386 meanR:392.7000 R:71.0000 loss:8.1610
Episode:4387 meanR:388.3700 R:67.0000 loss:4.9665
Episode:4388 meanR:384.0300 R:66.0000 loss:7.9575
Episode:4389 meanR:379.6700 R:64.0000 loss:6.3976
Episode:4390 meanR:375.3000 R:63.0000 loss:3.7749
Episode:4391 meanR:370.9100 R:61.0000 loss:3.9197
Episode:4392 meanR:366.5000 R:59.0000 loss:6.3510
Episode:4393 meanR:362.1500 R:65.0000 loss:4.9962
Episode:4394 meanR:357.7900 R:64.0000 loss:6.5642
Episode:4395 meanR:353.4100 R:62.0000 loss:4.1576
Episode:4396 meanR:349.0600 R:65.0000 loss:5.2329
Episode:4397 meanR:344.7100 R:65.0000 loss:4.3212
Episode:4398 meanR:340.3900 R:68.0000 loss:4.4708
Episode:4399 meanR:336.0000 R:61.0000 loss:3.0814
Episode:4400 meanR:331.6800 R:68.0000 loss:4.

Episode:4542 meanR:167.1700 R:13.0000 loss:16.2380
Episode:4543 meanR:165.3800 R:13.0000 loss:14.6818
Episode:4544 meanR:164.0300 R:12.0000 loss:13.8380
Episode:4545 meanR:162.5300 R:13.0000 loss:12.9515
Episode:4546 meanR:161.1900 R:14.0000 loss:13.4686
Episode:4547 meanR:159.9300 R:13.0000 loss:11.8952
Episode:4548 meanR:158.5900 R:14.0000 loss:13.3830
Episode:4549 meanR:157.1300 R:13.0000 loss:13.4503
Episode:4550 meanR:155.7000 R:14.0000 loss:13.6776
Episode:4551 meanR:153.8400 R:15.0000 loss:11.9896
Episode:4552 meanR:148.9700 R:13.0000 loss:11.6787
Episode:4553 meanR:144.1100 R:14.0000 loss:10.3052
Episode:4554 meanR:139.2600 R:15.0000 loss:12.2855
Episode:4555 meanR:134.4400 R:18.0000 loss:13.8660
Episode:4556 meanR:129.6100 R:17.0000 loss:11.5789
Episode:4557 meanR:124.7500 R:14.0000 loss:11.2845
Episode:4558 meanR:119.9000 R:15.0000 loss:12.5750
Episode:4559 meanR:115.0400 R:14.0000 loss:12.8114
Episode:4560 meanR:110.1700 R:13.0000 loss:13.2304
Episode:4561 meanR:109.3800 R:1

Episode:4707 meanR:12.4400 R:11.0000 loss:9.0848
Episode:4708 meanR:12.4300 R:10.0000 loss:8.2322
Episode:4709 meanR:12.4000 R:11.0000 loss:10.0086
Episode:4710 meanR:12.3800 R:12.0000 loss:8.2809
Episode:4711 meanR:12.3500 R:10.0000 loss:7.9286
Episode:4712 meanR:12.3300 R:10.0000 loss:7.2477
Episode:4713 meanR:12.3000 R:10.0000 loss:6.5588
Episode:4714 meanR:12.3200 R:12.0000 loss:6.3616
Episode:4715 meanR:12.3000 R:12.0000 loss:9.0046
Episode:4716 meanR:12.3100 R:13.0000 loss:7.5903
Episode:4717 meanR:12.2900 R:12.0000 loss:6.3901
Episode:4718 meanR:12.3100 R:13.0000 loss:5.7570
Episode:4719 meanR:12.2900 R:10.0000 loss:6.6736
Episode:4720 meanR:12.3100 R:13.0000 loss:8.4671
Episode:4721 meanR:12.3000 R:12.0000 loss:8.6443
Episode:4722 meanR:12.2700 R:11.0000 loss:8.8320
Episode:4723 meanR:12.2600 R:11.0000 loss:7.4021
Episode:4724 meanR:12.2500 R:12.0000 loss:8.6516
Episode:4725 meanR:12.2700 R:15.0000 loss:9.1609
Episode:4726 meanR:12.2700 R:12.0000 loss:10.3150
Episode:4727 meanR

Episode:4873 meanR:11.3100 R:11.0000 loss:21.9713
Episode:4874 meanR:11.3300 R:12.0000 loss:23.4977
Episode:4875 meanR:11.3300 R:12.0000 loss:20.8306
Episode:4876 meanR:11.3200 R:11.0000 loss:22.8206
Episode:4877 meanR:11.3300 R:12.0000 loss:21.4196
Episode:4878 meanR:11.3300 R:12.0000 loss:25.0917
Episode:4879 meanR:11.3300 R:11.0000 loss:24.4942
Episode:4880 meanR:11.3400 R:12.0000 loss:21.4562
Episode:4881 meanR:11.3300 R:11.0000 loss:19.4424
Episode:4882 meanR:11.3200 R:11.0000 loss:18.5729
Episode:4883 meanR:11.3200 R:13.0000 loss:24.2374
Episode:4884 meanR:11.3400 R:12.0000 loss:18.7634
Episode:4885 meanR:11.3500 R:12.0000 loss:22.2990
Episode:4886 meanR:11.3500 R:11.0000 loss:22.0332
Episode:4887 meanR:11.3500 R:12.0000 loss:18.4540
Episode:4888 meanR:11.3500 R:12.0000 loss:20.6192
Episode:4889 meanR:11.3600 R:11.0000 loss:20.7354
Episode:4890 meanR:11.3700 R:11.0000 loss:19.1184
Episode:4891 meanR:11.3800 R:12.0000 loss:21.4904
Episode:4892 meanR:11.3800 R:11.0000 loss:20.9061


Episode:5034 meanR:151.5600 R:9.0000 loss:117.7085
Episode:5035 meanR:151.5300 R:10.0000 loss:128.3759
Episode:5036 meanR:156.4100 R:500.0000 loss:142.4926
Episode:5037 meanR:156.3900 R:9.0000 loss:202.2130
Episode:5038 meanR:156.4400 R:15.0000 loss:229.2505
Episode:5039 meanR:161.3100 R:500.0000 loss:156.2317
Episode:5040 meanR:161.3500 R:16.0000 loss:182.4472
Episode:5041 meanR:166.2500 R:500.0000 loss:136.1874
Episode:5042 meanR:166.2100 R:10.0000 loss:166.2945
Episode:5043 meanR:171.1100 R:500.0000 loss:122.1448
Episode:5044 meanR:171.1100 R:13.0000 loss:209.2115
Episode:5045 meanR:171.1000 R:9.0000 loss:233.0538
Episode:5046 meanR:176.0000 R:500.0000 loss:187.8890
Episode:5047 meanR:180.8700 R:500.0000 loss:113.1528
Episode:5048 meanR:180.8100 R:9.0000 loss:103.6524
Episode:5049 meanR:185.6900 R:500.0000 loss:105.1871
Episode:5050 meanR:185.6800 R:12.0000 loss:132.5906
Episode:5051 meanR:189.9600 R:500.0000 loss:120.7551
Episode:5052 meanR:192.3500 R:500.0000 loss:101.0373
Episode

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(loss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Average losses')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episode/epoch
    for _ in range(10):
        total_reward = 0
        state = env.reset()
        initial_state = sess.run(model.initial_state) # Qs or current batch or states[:-1]
        
        # Steps/batches
        while True:
            env.render()
            action_logits, initial_state = sess.run([model.actions_logits, model.final_state],
                                                    feed_dict = {model.states: state.reshape([1, -1]), 
                                                                 model.initial_state: initial_state})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                break
        # At the end of each episode
        print('total_reward:{}'.format(total_reward))

# Close the env
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward:500.0
total_reward:500.0
total_reward:500.0
total_reward:500.0
total_reward:500.0
total_reward:500.0
total_reward:500.0
total_reward:500.0
total_reward:500.0
total_reward:500.0


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.